In [1]:
import os
from analysis.models import EncodeDataset
from collections import defaultdict

In [3]:
EncodeDataset.objects.count()

4506

In [4]:
#fn = os.path.abspath('./data/private/bigWigFiles.txt')
fn = os.path.abspath('data/bigWigFiles.txt')
assert os.path.exists(fn)

Using list instead of dict because of duplicate file names.

In [5]:
def getFileLocationList(fn):
    files = open(fn, 'r').readlines()
    filenames = [f.strip() for f in files]
    fns = []
    for file_ in filenames:
        #fns[os.path.basename(file_)] = file_
        fns.append({"base_name":os.path.basename(file_), "full_path":file_})
    return fns

#fns = getFileLocationDict(fn)
fns = getFileLocationList(fn)

In [6]:
len(fns)

5613

There are files with no identifying md5sum values in metadata.  However, none of these files have duplicated file names.  This likely needs to be checked when additional ENCODE files are added or the ENCODE data is updated.

In [8]:
for ds in EncodeDataset.objects.all():
    if ds.data_ambiguous:        
        if "ambig_md5sum" not in ds.extra_content:
            count = 0
            for entry_ in fns:
                if os.path.basename(ds.data_ambiguous.path) == entry_["base_name"]:
                    count += 1
            if count > 1:
                print("Duplicate file name exists with no md5sum!!")
    if ds.data_plus:
        #assert "plus_md5sum" in ds.extra_content
        if "plus_md5sum" not in ds.extra_content:
            count = 0
            for entry_ in fns:
                if os.path.basename(ds.data_plus.path) == entry_["base_name"]:
                    count += 1
            if count > 1:
                print("Duplicate file name exists with no md5sum!!")
    if ds.data_minus:
        #assert "minus_md5sum" in ds.extra_content
        if "minus_md5sum" not in ds.extra_content:
            count = 0
            for entry_ in fns:
                if os.path.basename(ds.data_minus.path) == entry_["base_name"]:
                    count += 1
            if count > 1:
                print("Duplicate file name exists with no md5sum!!")

In [9]:
md5sum_fn = os.path.abspath('data/md5list.txt')
assert os.path.exists(md5sum_fn)

def getMd5SumDict(fn):
    lines = open(fn, 'r').readlines()
    md5sums = []
    for line_ in lines:
        #fns[os.path.basename(file_)] = file_
        md5sums.append({"md5sum":line_.split()[0], "full_path":line_.strip().split()[1]})
    return md5sums

md5sums = getMd5SumDict(md5sum_fn)

In [10]:
len(md5sums)

5613

In [11]:
print(md5sums[0])

{'md5sum': 'b8d2f64650efaf479de2e5400a730a9c', 'full_path': '/apps/encodeTracks/mm9/wgEncodeCaltechHist/wgEncodeCaltechHistC2c12Ab2621FCntrl50bPcr1xSigRep1.bigWig'}


In [13]:
def findMatchingFilePath(file_name, file_md5sum, fns, md5sums):
    #file_name = os.path.basename(ds.data_ambiguous.path)
    if file_md5sum:
        for entry_ in md5sums:
            if file_md5sum == entry_["md5sum"]:
                return entry_["full_path"]
    else:
        matching_files = []
        for entry_ in fns:
            if file_name == entry_["base_name"]:
                matching_files.append(entry_["full_path"])
        if len(matching_files) == 1:
            return matching_files[0]
        else:
            return False

for ds in EncodeDataset.objects.all():
    
    if ds.data_ambiguous:
        if "ambig_md5sum" in ds.extra_content:
            ds.data_ambiguous = findMatchingFilePath(os.path.basename(ds.data_ambiguous.path), ds.extra_content["ambig_md5sum"], fns, md5sums)
        else:
            ds.data_ambiguous = findMatchingFilePath(os.path.basename(ds.data_ambiguous.path), False, fns, md5sums)
    if ds.data_plus:
        #ds.data_plus = findMatchingFilePath(os.path.basename(ds.data_plus.path), ds.extra_content["plus_md5sum"], fns, md5sums)
        if "plus_md5sum" in ds.extra_content:
            ds.data_plus = findMatchingFilePath(os.path.basename(ds.data_plus.path), ds.extra_content["plus_md5sum"], fns, md5sums)
        else:
            ds.data_plus = findMatchingFilePath(os.path.basename(ds.data_plus.path), False, fns, md5sums)
    if ds.data_minus:
        #ds.data_minus = findMatchingFilePath(os.path.basename(ds.data_minus.path), ds.extra_content["minus_md5sum"], fns, md5sums)
        if "minus_md5sum" in ds.extra_content:
            ds.data_ambiguous = findMatchingFilePath(os.path.basename(ds.data_minus.path), ds.extra_content["minus_md5sum"], fns, md5sums)
        else:
            ds.data_ambiguous = findMatchingFilePath(os.path.basename(ds.data_minus.path), False, fns, md5sums)
        
    #ds.save()

In [ ]:
def checkFile(file_name):
    match_count = 0
    for entry_ in fns:
        if file_name == entry_["full_path"]:
            match_count += 1
    if match_count != 1:
        print("Full path not present in file list!!")

for ds in EncodeDataset.objects.all():
    if ds.data_ambiguous:
        checkFile(ds.data_ambiguous)
    if ds.data_plus:
        checkFile(ds.data_plus)
    if ds.data_minus:
        checkFile(ds.data_minus)